In [3]:
import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import display
import time
import jax
from jax.random import multivariate_normal
import numpy as np
import plotly.graph_objects as go

var_X = widgets.FloatSlider(description='var_X',value=1.5,step=0.1,continuous_update=False)
var_Y = widgets.FloatSlider(description='var_Y',value=1.0,step=0.1,continuous_update=False)
mean_X = widgets.FloatSlider(description='mean_X',value=0,step=0.1,continuous_update=False)
mean_Y = widgets.FloatSlider(description='mean_Y',value=0,step=0.1,continuous_update=False)
cov_XY = widgets.FloatSlider(description='cov_XY',value=0.4,step=0.1,continuous_update=False)
sample_size = widgets.IntSlider(description='sample_size',min=5,max=20,continuous_update=False)

slider_dict = {"var_X":var_X,
               "var_Y":var_Y,
               "mean_X":mean_X,
               "mean_Y":mean_Y,
               "cov_XY":cov_XY,
               "sample_size":sample_size}

key = jax.random.PRNGKey(0)
cov = np.array([[var_X.value, cov_XY.value], [cov_XY.value, var_Y.value]])
mean = np.array([mean_X.value,mean_Y.value])
x1,x2 = multivariate_normal(key, mean, cov, (sample_size.value,)).T

fig_XY = go.Scatter3d(
    x=x1,
    y=x2,
    z=sample_size.value*[0],
    mode='markers',
    marker=dict(
        size=2,
        # color=z,                # set color to an array/list of desired values
        colorscale='Viridis',   # choose a colorscale
        opacity=0.8
    ))


fig = go.FigureWidget(data=[fig_XY])

fig.update_layout(margin=dict(l=0, r=0, b=0, t=0),
                    scene = dict(
                      #  xaxis = dict(nticks=4, range=[-100,100],),
                      #  yaxis = dict(nticks=4, range=[-50,100],),
                       zaxis = dict(nticks=4, range=[0,10],),),
                    )


container_slider = widgets.VBox(list(slider_dict.values()))

def response(v):
    ## DEBUG : Remove before sending
    print(var_X.value,var_Y.value,mean_X.value,mean_Y.value,cov_XY.value,sample_size.value)

    key = jax.random.PRNGKey(0)
    cov = np.array([[var_X.value, cov_XY.value], [cov_XY.value, var_Y.value]])
    mean = np.array([mean_X.value,mean_Y.value])
    x1,x2 = multivariate_normal(key, mean, cov, (sample_size.value,)).T

    with fig.batch_update():
        fig.data[0].x = x1
        fig.data[0].y = x2
        fig.data[0].z = sample_size.value*[0]
        


for slider in slider_dict.values():
    slider.observe(response, names='value')

widgets.VBox([container_slider,fig])